In [9]:
import pandas as pd
import numpy as np
import itertools
from datetime import datetime,timedelta,date

In [2]:
df = pd.read_csv('../features/clear_data/rm_features.csv',';',infer_datetime_format=True,index_col=['time'],parse_dates=['time'])

In [3]:
host_cons=[(host,group) for host,group in df.drop_duplicates(['host','consumer_group'])[['host','consumer_group']].values]


,host,consumer_group,cpu,dbtime,iowait,oth_wait,pga,pxenq,pxqry,pxqtime,...,sys_pga,sys_pxqct,sys_pxruns,sys_pxsrvt,sys_read,sys_redo,sys_rmwait,sys_syspct,sys_ucalls,sys_write
time,,,,,,,,,,,,,,,,,,,,,
2018-11-02 00:00:00,1,adw_unload_group,0.000000,0.000000,0.000000,0.000000,0.466667,0.000000,0.000000,0.000000,...,13020.866667,9.666667,257.866667,117.133333,3439.033333,21.700000,7.733333,8.533333,73.600000,130.466667
2018-11-02 01:00:00,1,adw_unload_group,3.066667,8.566667,1.266667,4.233333,707.933333,0.666667,12.000000,0.000000,...,13892.700000,14.300000,703.000000,187.600000,3196.366667,25.100000,3.500000,11.366667,83.266667,156.800000
2018-11-02 02:00:00,1,adw_unload_group,15.900000,19.933333,1.200000,2.833333,8088.933333,0.000000,26.000000,0.000000,...,26981.233333,12.200000,746.233333,210.500000,6799.800000,25.566667,1.400000,12.200000,98.466667,149.333333
2018-11-02 03:00:00,1,adw_unload_group,10.733333,14.200000,1.233333,2.233333,9257.733333,5.666667,20.000000,0.266667,...,33239.200000,8.633333,219.400000,167.966667,1926.866667,26.866667,0.266667,8.566667,108.500000,208.633333
2018-11-02 04:00:00,1,adw_unload_group,0.033333,0.066667,0.000000,0.033333,159.500000,0.000000,0.333333,0.000000,...,37467.466667,3.433333,259.166667,49.966667,1136.066667,37.400000,0.300000,8.200000,141.933333,399.200000


In [12]:
df.columns

Index(['host', 'consumer_group', 'cpu', 'dbtime', 'iowait', 'oth_wait', 'pga',
       'pxenq', 'pxqry', 'pxqtime', 'pxrel', 'read', 'reqs', 'write', 'yields',
       'max_utilization_limit', 'mgmt_p1', 'parallel_degree_limit_p1',
       'parallel_target_percentage', 'sys_actsess_avg', 'sys_actsess_max',
       'sys_cpu', 'sys_dbtime', 'sys_iowait', 'sys_oswait', 'sys_pga',
       'sys_pxqct', 'sys_pxruns', 'sys_pxsrvt', 'sys_read', 'sys_redo',
       'sys_rmwait', 'sys_syspct', 'sys_ucalls', 'sys_write'],
      dtype='object')

In [ ]:
train_period=2 # количество дней для обучения
eval_size=3 # количество прогнозов для теста
n_length=24 # количество часов
size=2 # параметр для ConvLSTM

column=['cpu_pct_maxul']

X_test,X_train,y_test,y_train=(defaultdict(list) for i in range(4))

def window_stack(a, pred_period=pred_period*n_length, train_period=train_period*n_length, rolling=1):
    window=int(len(a) - (pred_period+train_period))
    X=np.array([np.split(a[i:train_period+i],size) if size>1 else a[i:train_period+i] for i in range(0,int(window+rolling-1),rolling)])
    y=np.array([a[train_period+i:train_period+i+pred_period] for i in range(0,int(window+rolling-1),rolling)])

    return X,y

for (host,consumer_group) in host_cons:
    cutoff_df=df[(df.host==host)&(df.consumer_group==consumer_group)][column]
    cutoff_date=cutoff_df.index.max()-timedelta(days=eval_size*pred_period)

    X_test[(host,consumer_group)],y_test[(host,consumer_group)]=window_stack(cutoff_df[cutoff_df.index>cutoff_date-timedelta(days=train_period)].values,rolling=n_length)

    X_train[(host,consumer_group)],y_train[(host,consumer_group)]=window_stack(cutoff_df[cutoff_df.index<=cutoff_date].values)
